# 05. PyTorch Going Modular

Going modular involves turning notebook code (from a Jupyter Notebook or Google Colab notebook) into a series of different Python scripts that offer similar functionality.

For example, we could turn our notebook code from a series of cells into the following Python files:

* `data_setup.py` - a file to prepare and download data if needed.  
* `engine.py` - a file containing various training functions.  
* `model_builder.py` or model.py - a file to create a PyTorch model.  
* `train.py` - a file to leverage all other files and train a target PyTorch model.
* `utils.py` - a file dedicated to helpful utility functions.


**Production code** is code that runs to offer a service to someone or something.